In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [ ]:
path='imdb.csv'
#D:\Jovian data\data analysis projects\IMDB\imdb.csv

In [ ]:
try:
    load_df = pd.read_csv(path,sep=';')
    where=load_df['Id'].iloc[-1]
    No_zeros=7-len(str(where))
    No_zeros
    where_final='0'*No_zeros+str(where)
    q0=int(where_final[0])
    q1=int(where_final[1])
    q2=int(where_final[2])
    q3=int(where_final[3])
    q4=int(where_final[4])
    q5=int(where_final[5])
    q6=int(where_final[6])
except FileNotFoundError:
  q0=0
  q1=0
  q2=0
  q3=0
  q4=0
  q5=0
  q6=1

In [ ]:
#load_df

NameError: name 'load_df' is not defined

In [ ]:
print(q0,q1,q2,q3,q4,q5,q6)

0 0 0 0 0 0 1


In [ ]:
def keywords(id):
    word=''
    key_url='https://www.imdb.com/title/tt'+id+'/keywords/'
    #https://www.imdb.com/title/tt'+id_+'/'
    response_keyword=requests.get(key_url,headers=headers)
    content_keyword=response_keyword.content
    soup_keyword=BeautifulSoup(content_keyword,'html.parser')
    keyword_elements=soup_keyword.find_all('a',class_='ipc-metadata-list-summary-item__t')
    for element in keyword_elements:
     word += ', ' + element.text
    return word[2:len(word)]
   

In [ ]:
def release_info(id):
    key_url='https://www.imdb.com/title/tt'+id+'/releaseinfo/'
    #https://www.imdb.com/title/tt'+id_+'/'
    response_release_info=requests.get(key_url,headers=headers)
    content_release_info=response_release_info.content
    soup_release_info=BeautifulSoup(content_release_info,'html.parser')
    #date=soup_release_info.find('span',class_='ipc-metadata-list-item__list-content-item')
    try:
     #if soup_release_info.find('article',class_='sc-8eb80a87-0 hQQXzj').text is not None: 
        # return 'NaN'
     date=soup_release_info.find('span',class_='ipc-metadata-list-item__list-content-item')
     return date.text    

    except:
       return 'NaN'  
       
    


In [ ]:
def runtime(id):
    key_url='https://www.imdb.com/title/tt'+id+'/technical/'
    #https://www.imdb.com/title/tt'+id_+'/'
    response_runtime=requests.get(key_url,headers=headers)
    content_runtime=response_runtime.content
    soup_runtime=BeautifulSoup(content_runtime,'html.parser')
    runtime=soup_runtime.find_all('span',class_='ipc-metadata-list-item__list-content-item--subText')
    color=soup_runtime.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    try:
     return runtime[0].text,color[1].text
    except:
        return 'NaN',color[1].text
           
 

In [ ]:
df={'Id':[],'Title':[],'Storyline':[],'Director':[],'Date':[],'Runtime':[],'Rating':[],'Color':[],'Country':[],'keywords':[]}

In [10]:
%%time
i=0
for q in range(q0,10):
    for w in range(q1,10):
        for e in range(q2,10):
            for r in range(q3,10):
                for t in range(q4,10):
                    for y in range(q5,10):
                        for u in range(q6,10):
                          try:  

                            id_=str(str(q)+str(w)+str(e)+str(r)+str(t)+str(y)+str(u))
                            w_url='https://www.imdb.com/title/tt'+id_+'/'
                            response=requests.get(w_url,headers=headers)
                            content=response.content
                            soup=BeautifulSoup(content,'html.parser')
                            Title=soup.find('span',class_='sc-afe43def-1 fDTGTb').text
                            try:
                             storyline=soup.find_all('div',class_='ipc-html-content-inner-div')[2].text
                            except:
                                storyline='NaN'

                            Director=soup.find_all('a',class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')[0].text
                            Date=release_info(id_)
                            Runtime,colr=runtime(id_)
                            Rating=(soup.find_all('span',class_='sc-bde20123-1 iZlgcd')[0]).text
                            country=soup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base')
                            df['Country'].append(country)
                            key_words=keywords(str(id_))
                            i=i+1
                            print(i,w_url)
                            df['Id'].append(id_)
                            df['Title'].append(Title)
                            df['Storyline'].append(storyline)
                            df['Director'].append(Director)
                            df['Date'].append(Date)
                            df['Runtime'].append(Runtime)
                            df['Color'].append(colr)
                            df['Rating'].append(Rating)
                            df['keywords'].append(key_words)
                            print(df)
                            if i==100:
                              main_df=pd.DataFrame(df)
                              save_df=pd.concat([load_df, main_df], ignore_index=True)
                              save_df.to_csv(path,index=False,sep=';',header=True)
                              i=0
                          except:
                            pass
                        break    
                
          
       


                        

In [ ]:
i

In [ ]:
main_df=pd.DataFrame(df)

NameError: name 'pd' is not defined